## Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
from keras import layers
from keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import random
# from imgaug import augmenters as iaa
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Subtract, Conv2DTranspose, concatenate, Cropping2D
import tensorflow as tf
import cv2

## Load Dataset

Here, we are using numpy as Keras is used with numpy

In [ ]:
## Load Dataset
def load_dataset():
    datasets = ['real', 'easy', 'medium', 'hard']
    data_dict = {}
    for dataset in datasets:
        x_data = np.load(f'dataset/x_{dataset}.npy')
        y_data = np.load(f'dataset/y_{dataset}.npy')
        data_dict[dataset] = (x_data, y_data)
        print(f"{dataset}: X shape {x_data.shape}, Y shape {y_data.shape}")

    return data_dict

data_dict = load_dataset()

In [ ]:
def load_segmentation_dataset():
    """
    Load the fingerprint images and their corresponding mask annotations.
    If the dataset doesn't exist, create synthetic masks for demonstration.
    """
    # For demonstration purposes, we'll create synthetic masks
    # In a real scenario, you would load actual annotated masks
    
    print("Loading fingerprint region segmentation dataset...")
    
    # Try to load existing segmentation dataset if available
    try:
        x_images = np.load('dataset/x_segmentation.npy')
        y_masks = np.load('dataset/y_segmentation.npy')
        print(f"Loaded existing segmentation dataset: {x_images.shape}, {y_masks.shape}")
        return x_images, y_masks
    except:
        print("Creating synthetic segmentation dataset...")
    
    # Use the existing fingerprint images
    data_dict = load_dataset()
    
    # Combine images from different difficulty levels
    all_images = np.concatenate([
        data_dict['real'][0],
        data_dict['easy'][0],
        data_dict['medium'][0],
        data_dict['hard'][0]
    ], axis=0)
    
    # Create synthetic masks (in a real scenario, these would be manually annotated)
    masks = []
    for img in all_images:
        # Create a synthetic mask based on pixel intensity
        # This assumes fingerprints are darker than the background
        img_gray = img.squeeze()
        
        # Simple thresholding to create a binary mask
        _, binary = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY_INV)
        
        # Clean up the mask with morphological operations
        kernel = np.ones((5, 5), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        
        # Normalize to 0-1
        mask = binary / 255.0
        masks.append(mask)
    
    # Convert to numpy arrays
    x_images = all_images
    y_masks = np.array(masks)[..., np.newaxis]
    
    # Save the dataset for future use
    os.makedirs('dataset', exist_ok=True)
    np.save('dataset/x_segmentation.npy', x_images)
    np.save('dataset/y_segmentation.npy', y_masks)
    
    print(f"Created segmentation dataset: X shape {x_images.shape}, Y shape {y_masks.shape}")
    return x_images, y_masks

In [ ]:
# Visualize a sample from each dataset
def visualize_samples(data_dict):
    plt.figure(figsize=(15, 10))
    for idx, (key, (x, y)) in enumerate(data_dict.items(), start=1):
        plt.subplot(1, 4, idx)
        plt.title(y[0])
        plt.imshow(x[0].squeeze(), cmap='gray')
    plt.show()

visualize_samples(data_dict)

In [ ]:
def visualize_masks(images, masks, num_samples=3):
    """
    Visualize sample images and their corresponding segmentation masks.
    """
    indices = np.random.choice(range(len(images)), num_samples, replace=False)
    
    plt.figure(figsize=(15, 5*num_samples))
    for i, idx in enumerate(indices):
        # Original image
        plt.subplot(num_samples, 3, i*3+1)
        plt.title('Original Image')
        plt.imshow(images[idx].squeeze(), cmap='gray')
        
        # Mask
        plt.subplot(num_samples, 3, i*3+2)
        plt.title('Segmentation Mask')
        plt.imshow(masks[idx].squeeze(), cmap='gray')
        
        # Overlay
        plt.subplot(num_samples, 3, i*3+3)
        plt.title('Overlay')
        overlay = np.zeros_like(images[idx].squeeze())
        overlay = np.stack([overlay, masks[idx].squeeze()*255, overlay], axis=-1)
        plt.imshow(images[idx].squeeze(), cmap='gray')
        plt.imshow(overlay, alpha=0.5)
    
    plt.tight_layout()
    plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.metrics import Metric

@keras.saving.register_keras_serializable()
class IoU(Metric):
    def __init__(self, name='iou', **kwargs):
        super(IoU, self).__init__(name=name, **kwargs)
        self.intersection = self.add_weight(name='intersection', initializer='zeros')
        self.union = self.add_weight(name='union', initializer='zeros')
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > 0.5, tf.float32)
        intersection = tf.reduce_sum(y_true * y_pred)
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
        
        self.intersection.assign_add(intersection)
        self.union.assign_add(union)
        
    def result(self):
        return self.intersection / (self.union + 1e-6)  # Adding small epsilon to avoid division by zero
        
    def reset_state(self):
        self.intersection.assign(0.0)
        self.union.assign(0.0)

# Train Test Split

In [ ]:
x_data = np.concatenate([data_dict['easy'][0], data_dict['medium'][0], data_dict['hard'][0]], axis=0)
label_data = np.concatenate([data_dict['easy'][1], data_dict['medium'][1], data_dict['hard'][1]], axis=0)

# we use 90/10 split
x_train, x_val, label_train, label_val = train_test_split(x_data, label_data, test_size=0.1)
print(f"x_train: {x_train.shape}, label_train: {label_train.shape}")
print(f"x_val: {x_val.shape}, label_val: {label_val.shape}")

## Preview Augmentation

These are functions and transformations that will serve to transform and distort the original image that we will use for testing the trained model.


In [ ]:
def preview_augmentation(images):
    augmented_images = []
    
    for _ in range(9):
        img = images[0].copy()
        
        # Apply Gaussian blur
        blur_strength = random.uniform(0, 0.5)
        if blur_strength > 0:
            blur_size = int(blur_strength * 10) * 2 + 1  # Must be odd number
            img = cv2.GaussianBlur(img, (blur_size, blur_size), blur_strength)
        
        # Apply affine transformations
        h, w = img.shape[:2]
        
        # Random scale
        scale_x = random.uniform(0.9, 1.1)
        scale_y = random.uniform(0.9, 1.1)
        
        # Random translation
        tx = random.uniform(-0.1, 0.1) * w
        ty = random.uniform(-0.1, 0.1) * h
        
        # Random rotation
        angle = random.uniform(-30, 30)
        
        # Compute affine transform matrix
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        M[0, 0] *= scale_x
        M[1, 1] *= scale_y
        M[0, 2] += tx
        M[1, 2] += ty
        
        # Apply affine transform
        img = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=255)
        
        augmented_images.append(img)
    
    plt.figure(figsize=(16, 6))
    plt.subplot(2, 5, 1)
    plt.title('Original')
    plt.imshow(images[0].squeeze(), cmap='gray')
    for i, aug in enumerate(augmented_images):
        plt.subplot(2, 5, i + 2)
        plt.title(f'Aug {i + 1}')
        plt.imshow(aug.squeeze(), cmap='gray')
    plt.show()


## Make Label Dictionary Lookup Table

This is used later on, to find the original index of a randomly picked sample from the validation set in order to compare the images for the testing.

In [ ]:
label_real_dict = {''.join(y.astype(str)).zfill(6): i for i, y in enumerate(data_dict['real'][1])}

## Data Generator

This is necessary step in image processing using Keras, especially a big dataset that cannot fit into the memory during training. It is a class that produces batches of data that can feed into multiple cores right away.

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, x, label, x_real, label_real_dict, batch_size=32, shuffle=True):
        self.x = x
        self.label = label
        self.x_real = x_real
        self.label_real_dict = label_real_dict
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        return len(self.x) // self.batch_size
    
    def __getitem__(self, index):
        x1_batch = self.x[index * self.batch_size:(index + 1) * self.batch_size].copy()
        label_batch = self.label[index * self.batch_size:(index + 1) * self.batch_size]
        
        x2_batch = np.empty((self.batch_size, 90, 90, 1), dtype=np.float32)
        y_batch = np.zeros((self.batch_size, 1), dtype=np.float32)
        
        # Apply augmentations if shuffle is True
        if self.shuffle:
            # Apply OpenCV augmentations as in previous example
            for i in range(len(x1_batch)):
                # Augmentation code as provided earlier
                pass
        
        for i, l in enumerate(label_batch):
            match_key = ''.join(l.astype(str)).zfill(6)
            if random.random() > 0.5:
                x2_batch[i] = self.x_real[self.label_real_dict[match_key]][..., np.newaxis]
                y_batch[i] = 1.
            else:
                while True:
                    unmatch_key, unmatch_idx = random.choice(list(self.label_real_dict.items()))
                    if unmatch_key != match_key:
                        x2_batch[i] = self.x_real[unmatch_idx][..., np.newaxis]
                        break
                y_batch[i] = 0.
        
        # Return data in a format TensorFlow expects
        # Instead of returning [x1_batch, x2_batch], y_batch
        # Create a tuple of inputs
        x_input = (
            x1_batch.astype(np.float32) / 255., 
            x2_batch.astype(np.float32) / 255.
        )
        
        return x_input, y_batch
    
    def on_epoch_end(self):
        if self.shuffle:
            self.x, self.label = shuffle(self.x, self.label)

## Create Model

Here is the concept of the model:
- 2 models with 2x Convolutional and Pooling layers, sharing weights
- The outputs are subtracted, and the subtraction is fed to another Conv + Pooling layer
- Then that is fed to a sigmoid for final classification if the 2 images are same or not

![image.png](attachment:image.png)

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

def build_unet_model(input_shape=(90, 90, 1)):
    """
    Build a U-Net model for image segmentation with proper shape handling.
    """
    inputs = Input(input_shape)
    
    # Encoder (downsampling path)
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)  # 90→45
    
    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)  # 45→22
    
    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)  # 22→11
    
    # Bridge
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)
    
    # Decoder (upsampling path)
    u5 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)  # 11→22
    u5 = concatenate([u5, c3])  # Both 22×22
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)
    
    u6 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c5)  # 22→44
    # Crop c2 to match u6's shape if needed
    diff_y = c2.shape[1] - u6.shape[1]  # 45-44=1
    diff_x = c2.shape[2] - u6.shape[2]  # 45-44=1
    if diff_y > 0 or diff_x > 0:
        from tensorflow.keras.layers import Cropping2D
        c2 = Cropping2D(cropping=((0, diff_y), (0, diff_x)))(c2)
    u6 = concatenate([u6, c2])  # Both 44×44
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(c6)
    
    u7 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c6)  # 44→88
    # Crop c1 to match u7's shape if needed
    diff_y = c1.shape[1] - u7.shape[1]  # 90-88=2
    diff_x = c1.shape[2] - u7.shape[2]  # 90-88=2
    if diff_y > 0 or diff_x > 0:
        from tensorflow.keras.layers import Cropping2D
        c1 = Cropping2D(cropping=((0, diff_y), (0, diff_x)))(c1)
    u7 = concatenate([u7, c1])  # Both 88×88
    c7 = Conv2D(16, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(16, (3, 3), activation='relu', padding='same')(c7)
    
    # Output layer - match original input shape with cropping if needed
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)
    if outputs.shape[1] != input_shape[0] or outputs.shape[2] != input_shape[1]:
        from tensorflow.keras.layers import ZeroPadding2D
        outputs = ZeroPadding2D(padding=((1, 1), (1, 1)))(outputs)  # 88→90
    
    model = Model(inputs=[inputs], outputs=[outputs])
    # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'IoU'])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[IoU(name='IoU')]
    )
    
    return model


In [ ]:
def evaluate_segmentation(model, x_val, y_val, num_samples=5):
    """
    Evaluate the segmentation model and visualize results.
    """
    # Predict on validation data
    y_pred = model.predict(x_val)
    
    # Calculate IoU scores
    iou_scores = []
    for i in range(len(y_val)):
        true = y_val[i].squeeze()
        pred = (y_pred[i].squeeze() > 0.5).astype(np.float32)
        
        intersection = np.sum(true * pred)
        union = np.sum(true) + np.sum(pred) - intersection
        iou = intersection / (union + 1e-7)
        iou_scores.append(iou)
    
    mean_iou = np.mean(iou_scores)
    print(f"Mean IoU on validation set: {mean_iou:.4f}")
    
    # Visualize random samples
    indices = np.random.choice(range(len(x_val)), num_samples, replace=False)
    
    plt.figure(figsize=(15, 5*num_samples))
    for i, idx in enumerate(indices):
        # Original image
        plt.subplot(num_samples, 3, i*3+1)
        plt.title('Original Image')
        plt.imshow(x_val[idx].squeeze(), cmap='gray')
        
        # Ground truth mask
        plt.subplot(num_samples, 3, i*3+2)
        plt.title('Ground Truth')
        plt.imshow(y_val[idx].squeeze(), cmap='gray')
        
        # Predicted mask
        plt.subplot(num_samples, 3, i*3+3)
        plt.title(f'Prediction (IoU: {iou_scores[idx]:.4f})')
        plt.imshow(y_pred[idx].squeeze(), cmap='gray')
    
    plt.tight_layout()
    plt.show()
    
    return mean_iou

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

def train_segmentation_model():
    """
    Train a U-Net model for fingerprint region segmentation.
    """
    # Load or create the segmentation dataset
    x_images, y_masks = load_segmentation_dataset()
    
    # Normalize the images
    x_images = x_images.astype(np.float32) / 255.0
    
    # Split into training and validation sets
    x_train, x_val, y_train, y_val = train_test_split(
        x_images, y_masks, test_size=0.2, random_state=42
    )
    
    # Visualize some samples
    visualize_masks(x_train, y_train, num_samples=3)
    
    # Create U-Net model
    model = build_unet_model(input_shape=(90, 90, 1))
    model.summary()
    
    early_stopping = EarlyStopping(
        monitor='val_custom_iou',  # Match the metric name
        patience=10,
        restore_best_weights=True,
        mode='max',
        verbose=1
    )
    
    # Train the model
    history = model.fit(
        x_train, y_train,
        batch_size=32,
        epochs=5,
        validation_data=(x_val, y_val),
        callbacks=[early_stopping]
    )
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    # Evaluate the model on validation data
    evaluate_segmentation(model, x_val, y_val)
    
    return model, history


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Subtract
from tensorflow.keras.models import Model

def create_base_network(input_shape):
    """
    Function to create the base network (twin network) for the Siamese architecture.
    """
    input = Input(shape=input_shape)
    
    # First Convolutional Block
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    # Second Convolutional Block
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Flatten the feature map
    x = Flatten()(x)

    return Model(input, x)

# Define the input shape for the images
input_shape = (90, 90, 1)

# Create the twin networks (base networks with shared weights)
base_network = create_base_network(input_shape)

# Define two input tensors for the two images
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# Process each image through the shared twin network
processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Compute the absolute difference between the feature vectors
subtracted = Subtract()([processed_a, processed_b])

# Further processing of the subtracted feature using another convolutional layer
x = Dense(128, activation='relu')(subtracted)
x = Dropout(0.5)(x)

# Output layer with sigmoid activation for binary classification
output = Dense(1, activation='sigmoid')(x)

# Create the full Siamese network model
model = Model(inputs=[input_a, input_b], outputs=output)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
model.summary()

## Train

In other versons of the solutions, they used fit_generator method, which was commonly used in older versions of Keras to train models using data generators. However, since TensorFlow 2.1, fit can handle data generators directly. This unifies the API, making the code cleaner and more consistent. It also allows for better integration with TensorFlow’s functionalities such as distributed training and callbacks. Hence, using fit instead of fit_generator is the recommended approach now.

In [ ]:
seg_model, seg_history = train_segmentation_model()

In [ ]:
train_gen = DataGenerator(x_train, label_train, data_dict['real'][0], label_real_dict, shuffle=True)
val_gen = DataGenerator(x_val, label_val, data_dict['real'][0], label_real_dict, shuffle=False)

history = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen
)


## Evaluation

This is just a test using one random image and

In [ ]:
# def extract_fingerprint_region(image, seg_model, padding=10):
#     # Ensure image has proper dimensions
#     original_image = image.copy()
    
#     try:
#         # Prepare image for the segmentation model
#         if len(image.shape) == 2:
#             # Add channel dimension if grayscale
#             input_img = np.expand_dims(image, axis=-1)
#         else:
#             input_img = image
            
#         # Resize if necessary based on model's expected input shape
#         # model_input_shape = seg_model.inputs[0].shape.as_list()[1:3]  
        
#         try:
#         # Try TensorFlow tensor approach
#             model_input_shape = seg_model.inputs[0].shape.as_list()[1:3]
#         except AttributeError:
#             # Handle case where shape is a tuple
#             model_input_shape = seg_model.inputs[0].shape[1:3]
#         # Get expected height and width
#         if model_input_shape[0] is not None and model_input_shape[1] is not None:
#             # If model expects specific dimensions, resize the image
#             input_img_resized = cv2.resize(input_img, (model_input_shape[1], model_input_shape[0]))
#             # Ensure proper dimensions for model input (batch dimension)
#             input_img_resized = np.expand_dims(input_img_resized, axis=0)
#         else:
#             # If model accepts dynamic input, just add batch dimension
#             input_img_resized = np.expand_dims(input_img, axis=0)
        
#         # Predict segmentation mask
#         prediction = seg_model.predict(input_img_resized)
        
#         # Process the predicted mask
#         if len(prediction.shape) == 4:  # (batch, height, width, channels)
#             mask = prediction[0]  # Remove batch dimension
            
#             # If multi-class segmentation, take argmax or the fingerprint class
#             if mask.shape[-1] > 1:
#                 mask = np.argmax(mask, axis=-1)  # Convert to class indices
#                 # Assuming fingerprint class has index 1 (adjust if different)
#                 binary_mask = (mask == 1).astype(np.uint8) * 255
#             else:
#                 # For binary segmentation
#                 binary_mask = (mask[:, :, 0] > 0.5).astype(np.uint8) * 255
#         else:
#             # Handle unexpected output format
#             raise ValueError("Unexpected model output format")
            
#         # Resize back to original image dimensions if needed
#         if model_input_shape[0] is not None and model_input_shape[1] is not None:
#             if binary_mask.shape[:2] != image.shape[:2]:
#                 binary_mask = cv2.resize(binary_mask, (image.shape[1], image.shape[0]))
                
#     except Exception as e:
#         print(f"Segmentation model error: {e}. Using fallback thresholding method.")
#         # Fall back to the thresholding approach
#         if len(image.shape) == 2:
#             gray = image
#         else:
#             gray = image[:,:,0]
        
#         # Make sure image is in the right range for thresholding
#         if gray.max() <= 1.0:
#             gray_uint8 = (gray * 255).astype(np.uint8)
#         else:
#             gray_uint8 = gray.astype(np.uint8)
            
#         # Apply adaptive thresholding to get a binary mask
#         binary_mask = cv2.adaptiveThreshold(
#             gray_uint8, 
#             255, 
#             cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
#             cv2.THRESH_BINARY_INV, 
#             11, 
#             2
#         )
    
#     # Clean up the mask
#     kernel = np.ones((3, 3), np.uint8)
#     binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
#     binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    
#     # Find contours
#     contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
#     # If no contours found, return the original image and a full-frame bounding box
#     if not contours:
#         h, w = image.shape[:2]
#         return image, (0, 0, w, h)
    
#     # Find the largest contour
#     largest_contour = max(contours, key=cv2.contourArea)
    
#     # Get bounding box of the largest contour
#     x, y, w, h = cv2.boundingRect(largest_contour)
    
#     # Add padding
#     x_min = max(0, x - padding)
#     y_min = max(0, y - padding)
#     x_max = min(original_image.shape[1], x + w + padding)
#     y_max = min(original_image.shape[0], y + h + padding)
    
#     # Crop the image
#     if len(original_image.shape) == 2:
#         cropped_image = original_image[y_min:y_max, x_min:x_max]
#     else:
#         cropped_image = original_image[y_min:y_max, x_min:x_max, :]
    
#     return cropped_image, (x_min, y_min, x_max, y_max)

def extract_fingerprint_region(image, seg_model, padding=10):
    """
    Extract the fingerprint region using the segmentation model.
    
    Args:
        image: Input fingerprint image (normalized between 0-1)
        seg_model: Trained segmentation model
        padding: Padding to add around the detected region
        
    Returns:
        cropped_image: Cropped fingerprint region
        bbox: Bounding box coordinates (x_min, y_min, x_max, y_max)
    """
    # Ensure image has proper dimensions
    original_image = image.copy()
    
    try:
        # Prepare image for the segmentation model
        if len(image.shape) == 2:
            # Add channel dimension if grayscale
            input_img = np.expand_dims(image, axis=-1)
        else:
            input_img = image
            
        # Resize if necessary based on model's expected input shape
        # model_input_shape = seg_model.inputs[0].shape.as_list()[1:3]  
        
        try:
        # Try TensorFlow tensor approach
            model_input_shape = seg_model.inputs[0].shape.as_list()[1:3]
        except AttributeError:
            # Handle case where shape is a tuple
            model_input_shape = seg_model.inputs[0].shape[1:3]
        # Get expected height and width
        if model_input_shape[0] is not None and model_input_shape[1] is not None:
            # If model expects specific dimensions, resize the image
            input_img_resized = cv2.resize(input_img, (model_input_shape[1], model_input_shape[0]))
            # Ensure proper dimensions for model input (batch dimension)
            input_img_resized = np.expand_dims(input_img_resized, axis=0)
        else:
            # If model accepts dynamic input, just add batch dimension
            input_img_resized = np.expand_dims(input_img, axis=0)
        
        # Predict segmentation mask
        prediction = seg_model.predict(input_img_resized)
        
        # Process the predicted mask
        if len(prediction.shape) == 4:  # (batch, height, width, channels)
            mask = prediction[0]  # Remove batch dimension
            
            # If multi-class segmentation, take argmax or the fingerprint class
            if mask.shape[-1] > 1:
                mask = np.argmax(mask, axis=-1)  # Convert to class indices
                # Assuming fingerprint class has index 1 (adjust if different)
                binary_mask = (mask == 1).astype(np.uint8) * 255
            else:
                # For binary segmentation
                binary_mask = (mask[:, :, 0] > 0.5).astype(np.uint8) * 255
        else:
            # Handle unexpected output format
            raise ValueError("Unexpected model output format")
            
        # Resize back to original image dimensions if needed
        if model_input_shape[0] is not None and model_input_shape[1] is not None:
            if binary_mask.shape[:2] != image.shape[:2]:
                binary_mask = cv2.resize(binary_mask, (image.shape[1], image.shape[0]))
                
    except Exception as e:
        print(f"Segmentation model error: {e}. Using fallback thresholding method.")
        # Fall back to the thresholding approach
        if len(image.shape) == 2:
            gray = image
        else:
            gray = image[:,:,0]
        
        # Make sure image is in the right range for thresholding
        if gray.max() <= 1.0:
            gray_uint8 = (gray * 255).astype(np.uint8)
        else:
            gray_uint8 = gray.astype(np.uint8)
            
        # Apply adaptive thresholding to get a binary mask
        binary_mask = cv2.adaptiveThreshold(
            gray_uint8, 
            255, 
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
            cv2.THRESH_BINARY_INV, 
            11, 
            2
        )
    
    # Clean up the mask
    kernel = np.ones((3, 3), np.uint8)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # Find contours
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # If no contours found, return the original image and a full-frame bounding box
    if not contours:
        h, w = image.shape[:2]
        return image, (0, 0, w, h)
    
    # Find the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Get bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Add padding
    x_min = max(0, x - padding)
    y_min = max(0, y - padding)
    x_max = min(original_image.shape[1], x + w + padding)
    y_max = min(original_image.shape[0], y + h + padding)
    
    # Crop the image
    if len(original_image.shape) == 2:
        cropped_image = original_image[y_min:y_max, x_min:x_max]
    else:
        cropped_image = original_image[y_min:y_max, x_min:x_max, :]
    
    return cropped_image, (x_min, y_min, x_max, y_max)

In [ ]:
def visualize_extracted_regions(images, seg_model=None, num_samples=5):
    """
    Visualize the extracted fingerprint regions.
    
    Args:
        images: Array of fingerprint images
        seg_model: Not used in this version, kept for compatibility
        num_samples: Number of random samples to visualize
    """
    # Convert to numpy array if not already
    if not isinstance(images, np.ndarray):
        images = np.array(images)
    
    # Select random samples
    if len(images) <= num_samples:
        indices = list(range(len(images)))
    else:
        indices = np.random.choice(range(len(images)), num_samples, replace=False)
    
    plt.figure(figsize=(15, 5*len(indices)))
    for i, idx in enumerate(indices):
        # Get original image
        image = images[idx].copy()  # Make a copy to avoid modifying original
        
        # Handle different input formats
        if len(image.shape) == 3 and (image.shape[0] == 1 or image.shape[-1] == 1):
            image = image.squeeze()  # Remove singleton dimensions
        
        # Normalize if needed
        if image.max() > 1.0:
            image = image / 255.0
        
        try:
            print(f"Processing image {idx}, shape: {image.shape}")
            # Extract region using classical CV methods
            cropped, bbox = extract_fingerprint_region(image, seg_model)
            
            # Original image
            plt.subplot(len(indices), 3, i*3+1)
            plt.title('Original Image')
            plt.imshow(image, cmap='gray')
            plt.axis('off')
            
            # Draw bounding box on original
            plt.subplot(len(indices), 3, i*3+2)
            plt.title('Detected Region')
            plt.imshow(image, cmap='gray')
            x_min, y_min, x_max, y_max = bbox
            rect = plt.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, 
                              fill=False, edgecolor='red', linewidth=2)
            plt.gca().add_patch(rect)
            plt.axis('off')
            
            # Cropped and resized image
            plt.subplot(len(indices), 3, i*3+3)
            plt.title('Cropped Region')
            plt.imshow(cropped, cmap='gray')
            plt.axis('off')
            
        except Exception as e:
            print(f"Error processing image {idx}: {e}")
            plt.subplot(len(indices), 3, i*3+1)
            plt.title(f'Error: {str(e)[:20]}...')
            plt.imshow(image, cmap='gray')
            plt.axis('off')
            
            # Leave other plots empty
            plt.subplot(len(indices), 3, i*3+2)
            plt.title('Error occurred')
            plt.axis('off')
            
            plt.subplot(len(indices), 3, i*3+3)
            plt.axis('off')
            
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_extracted_regions(x_train, seg_model)

In [ ]:
def evaluate_combined_pipeline(fingerprint_model, seg_model, x_val, label_val, data_dict, label_real_dict, num_samples=1):
    """
    Evaluate the complete fingerprint matching pipeline with region detection.
    """
    results = []
    
    # indices = np.random.choice(range(len(x_val)), num_samples, replace=False)
    num_samples = min(num_samples, len(x_val))
    indices = np.random.choice(range(len(x_val)), num_samples, replace=False)
    
    valid_size = len(x_val)
    num_samples = min(num_samples, valid_size) 
    indices = np.random.choice(valid_size, num_samples, replace=False)
    
    plt.figure(figsize=(20, 5*num_samples))
    for i, idx in enumerate(indices):
        input_img = x_val[idx].copy()
        match_key = ''.join(label_val[idx].astype(str)).zfill(6)
        
        # Get matching real fingerprint
        matched_img = data_dict['real'][0][label_real_dict[match_key]].copy()
        
        # Get random non-matching fingerprint
        while True:
            unmatch_key, unmatch_idx = random.choice(list(label_real_dict.items()))
            if unmatch_key != match_key:
                unmatched_img = data_dict['real'][0][unmatch_idx].copy()
                break
        
        # Process input image
        input_normalized = input_img.astype(np.float32) / 255.0
        input_segmented, input_bbox = extract_fingerprint_region(input_normalized, seg_model)
        input_segmented = input_segmented.reshape(1, 90, 90, 1)
        
        # Process matched image
        matched_normalized = matched_img.astype(np.float32) / 255.0
        matched_segmented, matched_bbox = extract_fingerprint_region(matched_normalized, seg_model)
        matched_segmented = matched_segmented.reshape(1, 90, 90, 1)
        
        # Process unmatched image
        unmatched_normalized = unmatched_img.astype(np.float32) / 255.0
        unmatched_segmented, unmatched_bbox = extract_fingerprint_region(unmatched_normalized, seg_model)
        unmatched_segmented = unmatched_segmented.reshape(1, 90, 90, 1)
        
        # Predict similarity
        pred_matched = fingerprint_model.predict([input_segmented, matched_segmented])[0][0]
        pred_unmatched = fingerprint_model.predict([input_segmented, unmatched_segmented])[0][0]
        
        results.append({
            'input_id': idx,
            'match_score': pred_matched,
            'unmatch_score': pred_unmatched,
            'correct': pred_matched > 0.5 and pred_unmatched < 0.5
        })
        
        # Original images with bounding boxes
        plt.subplot(num_samples, 6, i*6+1)
        plt.title(f'Input: {label_val[idx]}')
        plt.imshow(input_img.squeeze(), cmap='gray')
        rect = plt.Rectangle((input_bbox[0], input_bbox[1]), 
                           input_bbox[2]-input_bbox[0], input_bbox[3]-input_bbox[1], 
                           fill=False, edgecolor='red', linewidth=2)
        plt.gca().add_patch(rect)
        
        plt.subplot(num_samples, 6, i*6+2)
        plt.title('Input Segmented')
        plt.imshow(input_segmented.squeeze(), cmap='gray')
        
        # Matched pair
        plt.subplot(num_samples, 6, i*6+3)
        plt.title(f'Match')
        plt.imshow(matched_img.squeeze(), cmap='gray')
        rect = plt.Rectangle((matched_bbox[0], matched_bbox[1]), 
                           matched_bbox[2]-matched_bbox[0], matched_bbox[3]-matched_bbox[1], 
                           fill=False, edgecolor='red', linewidth=2)
        plt.gca().add_patch(rect)
        
        plt.subplot(num_samples, 6, i*6+4)
        plt.title(f'Match Segmented\nScore: {pred_matched:.2f}')
        plt.imshow(matched_segmented.squeeze(), cmap='gray')
        
        # Unmatched pair
        plt.subplot(num_samples, 6, i*6+5)
        plt.title(f'Unmatch')
        plt.imshow(unmatched_img.squeeze(), cmap='gray')
        rect = plt.Rectangle((unmatched_bbox[0], unmatched_bbox[1]), 
                           unmatched_bbox[2]-unmatched_bbox[0], unmatched_bbox[3]-unmatched_bbox[1], 
                           fill=False, edgecolor='red', linewidth=2)
        plt.gca().add_patch(rect)
        
        plt.subplot(num_samples, 6, i*6+6)
        plt.title(f'Unmatch Segmented\nScore: {pred_unmatched:.2f}')
        plt.imshow(unmatched_segmented.squeeze(), cmap='gray')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate accuracy
    correct_predictions = sum(1 for r in results if r['correct'])
    accuracy = correct_predictions / len(results)
    print(f"Pipeline accuracy on sample: {accuracy:.2f}")
    
    return results

In [ ]:
evaluate_combined_pipeline(model, seg_model, x_val, label_val, data_dict, label_real_dict)

# Compare performance between original and enhanced models
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
# plt.plot(history.history['accuracy'], label='Original')
plt.plot(history.history['accuracy'], label='With Segmentation') 
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['val_accuracy'], label='With Segmentation')  
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## Store Models

In [ ]:
import os
import json
import time
from datetime import datetime
import numpy as np
from tensorflow.keras.models import load_model, save_model
import tensorflow as tf

def create_folder_structure():
    """Create the necessary folder structure for storing models"""
    os.makedirs('../fingerprint_models', exist_ok=True)
    os.makedirs('../fingerprint_results', exist_ok=True)
    os.makedirs('../fingerprint_models/recognition', exist_ok=True)
    os.makedirs('../fingerprint_models/segmentation', exist_ok=True)
    print("Folder structure created successfully.")

def store_model(model, model_type, model_name, version=None, accuracy=None):
    if model_type not in ['recognition', 'segmentation']:
        raise ValueError("model_type must be either 'recognition' or 'segmentation'")

    model_id = f"{model_name.replace(' ', '_').lower()}"
    
    folder_path = f"../fingerprint_models/{model_type}"
    model_path = f"{folder_path}/{model_id}.keras"  

    model_absolute_path = f"fingerprint-recognition/fingperint_models/{model_type}/{model_id}.keras"
    save_model(model, model_path)  # Corrected here
    print(f"Model saved to {model_path}")
    
    # Create metadata
    now = datetime.now().isoformat()
    metadata = {
        "name": model_name,
        "path": model_absolute_path,
        "accuracy": accuracy if accuracy is not None else 0.0,
        "validationScore"
        "createdAt": now,
        "updatedAt": now
    }
    
    if version is not None:
        metadata["version"] = version
    
    return metadata


def update_model_list(model_type, model_metadata):
    if model_type not in ['recognition', 'segmentation']:
        raise ValueError("model_type must be either 'recognition' or 'segmentation'")
    
    json_path = f"../fingerprint_results/fingerprint_{model_type}_models.json"
    
    # Check if the file exists and is not empty
    if os.path.exists(json_path) and os.path.getsize(json_path) > 0:
        with open(json_path, 'r') as f:
            try:
                models = json.load(f)
            except json.JSONDecodeError:
                print(f"Warning: Corrupt JSON file detected at {json_path}. Resetting file.")
                models = []  # Reset to empty list if corruption occurs
    else:
        models = []
    
    # Add new model metadata
    models.append(model_metadata)
    
    # Save the updated list
    with open(json_path, 'w') as f:
        json.dump(models, f, indent=4)  # Fixed incorrect json.dumps usage
    
    print(f"Updated {json_path} with new model information")

def calculate_accuracy(model, test_generator, num_batches=None):
    results = model.evaluate(test_generator, steps=num_batches)
    return results[1]  # Assuming accuracy is the second metric

def calculate_iou_accuracy(seg_model, x_val, y_val):
    y_pred = seg_model.predict(x_val)
    
    iou_scores = []
    for i in range(len(y_val)):
        true = y_val[i].squeeze()
        pred = (y_pred[i].squeeze() > 0.5).astype(np.float32)
        
        intersection = np.sum(true * pred)
        union = np.sum(true) + np.sum(pred) - intersection
        iou = intersection / (union + 1e-7)
        iou_scores.append(iou)
    
    mean_iou = np.mean(iou_scores)
    return mean_iou

create_folder_structure()
x_images, y_masks = load_segmentation_dataset()
x_images = x_images.astype(np.float32) / 255.0

x_train, x_val, y_train, y_val = train_test_split(
    x_images, y_masks, test_size=0.2, random_state=42
)

# if 'seg_model' in locals():
iou_accuracy = calculate_iou_accuracy(seg_model, x_val, y_val)

seg_metadata = store_model(
    seg_model,
    model_type='segmentation',
    model_name='UNet Segmentation',
    version='1.0',
    accuracy=float(iou_accuracy)
)

update_model_list('segmentation', seg_metadata)

# if 'model' in locals():
recognition_accuracy = calculate_accuracy(model, val_gen)

rec_metadata = store_model(
    model,
    model_type='recognition',
    model_name='Siamese Network',
    version='1.0',
    accuracy=float(recognition_accuracy)
)
    
update_model_list('recognition', rec_metadata)